# 加载库函数及导入文件

In [2]:
%load_ext autoreload
%autoreload 2
import warnings
import pathlib
from corelibs.config import *
warnings.filterwarnings('ignore', message="Workbook contains no default style, apply openpyxl's default", category=UserWarning)
conf_data = load_conf()
OUTPUT_DIR_PATH = pathlib.Path(r"J:\银行分析结果")
OUTPUT_DIR_PATH = pathlib.Path(r"D:\gitcodes\testdata\银行分析结果test")
OUTPUT_DIR_PATH

WindowsPath('D:/gitcodes/testdata/银行分析结果test')

In [ ]:
conf_data

# 测试

## 提取excel表头和md5值

In [5]:
from hashlib import md5
acc_file = pathlib.Path(r"J:\杨孟军\银行\工商银行\1\陶瑞兰120221196603113061\1.xls")
acc_file = pathlib.Path(r"J:\杨孟军\银行\工商银行\1\陶瑞兰120221196603113061\2.xls")
print(a := read_header(acc_file, header=0))
md5(a.encode()).hexdigest()

['账号', '币种', '卡号', '交易时间戳', '工作日期', '借贷标志', '发生额', '余额', '注释', '对方帐户', '对方客户编号', '交易地区号', '交易地区号(原始)', '交易网点号', '交易网点号(原始)', '帐户物理网点号', '柜员号', '授权柜员号', '交易代码', '服务界面', '交易日期', '记帐时间', '零售存取款摘要', '钞汇标志', '终端号', '交易场所简称', '对方户名', '对方开户行名\t']


'0b02a413bd3c59d74a82e3fd3909c759'

## 测试yaml配置文件语法，及内部操作指令对象转换

In [33]:
load_conf()
print(get_output_format())
print(len([print(x) for x in get_header_hash().items()]))

excel
('6d3d0d1e252f43a000010e19d8b2dd57', ['天津银行', '账户'])
('7509cd85808831c7004c8c78e9267d7c', ['天津银行', '流水'])
('5ff3970075646f3a7fffdef5430539d7', ['建设银行网点', '账户'])
('8fa9c0776b857793f91335192398aeba', ['建设银行网点', '流水'])
('5b70f468037b3cfe76bace72751f4e3e', ['工商银行网点', '账户'])
('0b02a413bd3c59d74a82e3fd3909c759', ['工商银行网点', '流水'])
6


In [ ]:
print(len([print(x) for x in get_conf_cache().items()]))

In [11]:
load_conf()
conf_obj = get_conf_obj('工商银行网点','流水')
[print(x) for x in conf_obj]

None
{'银行': '工商银行'}
{'账号': True, '余额': True, '借贷标志': True, '发生额': True, '交易日期': True}
{'服务界面': '备注', '注释': '摘要', '对方帐户': '对方账号', '对方开户行名': '对方开户行'}
{'交易机构': ['交易地区号', '交易网点号']}
{'日期': ['交易日期', ' %Y-%m-%d']}
{'时间': ['记帐时间', ' %H.%M.%S']}
{'余额': True, '发生额': True}
{'C': '借', 'CD_col': '借贷标志', 'C_col': '出账金额', 'trans_col': '发生额', 'D': '贷', 'D_col': '入账金额'}
{}
['银行', '日期', '时间', '姓名', '账号', '卡号', '备注', '摘要', '币种', '出账金额', '入账金额', '余额', '交易机构', '对方户名', '对方账号', '对方开户行', '借贷标志', '发生额', '交易日期', '交易时间戳', '交易日期', '工作日期', '记帐时间', '借贷标志', '发生额', '交易地区号', '交易地区号(原始)', '交易网点号', '交易网点号(原始)', '对方客户编号', '帐户物理网点号', '柜员号', '授权柜员号', '交易代码', '零售存取款摘要', '钞汇标志', '终端号', '交易场所简称']
{'姓名': ['账号', '姓名', '账号', '姓名']}


[None, None, None, None, None, None, None, None, None, None, None, None]

##  测试单个账户文件读取及储存

In [26]:
from corelibs.process import process_account_file_general
acc_file = pathlib.Path(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】\1080860401675667712130255_3_1_watermark.xlsx")
df_acc = process_account_file_general(acc_file, OUTPUT_DIR_PATH,'建设银行网点', '账户')
df_acc

,银行,姓名,证件号码,电话,地址,工作单位,账号,卡号,币种,余额,...,结算账户分类代码,可售产品编号,可售产品名称,证件类型,客户编号,册号,笔号,开户机构号,开户柜员号,销户机构号
0,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0061759980130631946,4367420061750483920,人民币元,0.00,...,一类账户,00000012,维萨磁条储蓄卡,居民身份证,479190000103765349,NaN,NaN,120675900,NaN,NaN
1,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0064659980130231346,4367420064650105775,人民币元,0.00,...,一类账户,00000012,维萨磁条储蓄卡,居民身份证,479190000103765349,NaN,NaN,120678901,NaN,NaN
2,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,14491.56,...,一类账户,00000012,银联复合理财金卡,居民身份证,479190000103765349,NaN,NaN,120835100,NaN,NaN
3,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,9000.00,...,其他账户,00000005,整存整取储蓄存款,居民身份证,479190000103765349,NaN,1,120835100,0,NaN
4,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,9000.00,...,其他账户,00000005,整存整取储蓄存款,居民身份证,479190000103765349,NaN,2,120835100,0,NaN
5,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,13969.93,...,其他账户,00000005,整存整取储蓄存款,居民身份证,479190000103765349,NaN,3,120835100,0,NaN
6,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,20000.00,...,其他账户,00000005,整存整取储蓄存款,居民身份证,479190000103765349,NaN,4,120835100,Z1990006,NaN
7,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0065089980130243116,6214990060119881,人民币元,0.00,...,其他账户,00000002,零存整取储蓄存款,居民身份证,479190000103765349,NaN,1,120835100,0,NaN
8,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,0060019980131146165,6227000060010578489,人民币元,0.00,...,一类账户,00000012,银联磁条公积金龙卡天津,居民身份证,479190000103765349,NaN,NaN,120615300,NaN,NaN
9,建设银行,夏澜晏,230405198805250326,18602621089,梅江街道卡梅尔天淳园18-5 天津生态城中天大道美林园11号楼1门502号,天津人力,1050819980130348640,6227001050819041200,人民币元,0.00,...,一类账户,00000012,银联磁条储蓄卡,居民身份证,479190000103765349,NaN,NaN,230645651,NaN,NaN


In [3]:
import dtale

In [4]:
dtale.show(df, open_browser=True)

## 测试单个流水文件的读取与储存

In [36]:
from corelibs.process import process_statment_file_general
acc_file = pathlib.Path(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】\1080860401675667712130255_4_1_watermark.xlsx")
process_statment_file_general(acc_file, OUTPUT_DIR_PATH,'建设银行网点', '流水','a', df_acc)

,银行,日期,时间,姓名,账号,卡号,备注,摘要,币种,出账金额,...,交易流水号,第三方订单号,交易渠道,自助设备编号,钞汇,册号,笔号,活期存款账户明细号,柜员号,交易机构号
0,建设银行,2012-07-11,09:57:04,夏澜晏,0061759980130631946,4367420061750483920,建行天津塘沽分行储,现金存入,人民币元,NaN,...,1206704460070000017,NaN,120670240092,NaN,钞,NaN,NaN,1,120670240092,120670446
1,建设银行,2012-07-12,17:44:03,夏澜晏,0061759980130631946,4367420061750483920,工资2012-7,自定义,人民币元,NaN,...,120080200D380001753,NaN,120080200D38,NaN,钞,NaN,NaN,2,120080200D38,120080200
2,建设银行,2012-07-12,17:45:06,夏澜晏,0061759980130631946,4367420061750483920,个人短信代扣,自定义,人民币元,24.00,...,120080200D380002131,NaN,120080200D38,NaN,钞,NaN,NaN,3,120080200D38,120080200
3,建设银行,2012-07-13,19:11:31,夏澜晏,0061759980130631946,4367420061750483920,建行天津塘沽分行储,ATM存款,人民币元,NaN,...,120670446A010000227,NaN,120670446A01,NaN,钞,NaN,NaN,4,120670446A01,120670446
4,建设银行,2012-07-14,21:02:57,夏澜晏,0061759980130631946,4367420061750483920,建行天津塘沽分行储,ATM取款,人民币元,500.00,...,120670446A010000161,NaN,120670446A01,NaN,钞,NaN,NaN,5,120670446A01,120670446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999,建设银行,2011-09-21,02:03:23,夏澜晏,3090439980120237054,6227003090430107012,0.00 0.00000,结息,人民币元,NaN,...,NaN,NaN,440999999001,NaN,钞,NaN,NaN,228,440999999001,NaN
2000,建设银行,2011-10-28,01:26:27,夏澜晏,3090439980120237054,6227003090430107012,NaN,账户管理费,人民币元,0.92,...,NaN,NaN,440999999001,NaN,钞,NaN,NaN,229,440999999001,NaN
2001,建设银行,2011-11-09,14:42:32,夏澜晏,3090439980120237054,6227003090430107012,鹤岗解放支行,ATM存款,人民币元,NaN,...,230645051A040000163,NaN,230645051A04,NaN,钞,NaN,NaN,230,230645051A04,230645051
2002,建设银行,2011-11-09,14:42:32,夏澜晏,3090439980120237054,6227003090430107012,鹤岗解放支行,手续费,人民币元,2.00,...,230645051A040000163,NaN,230645051A04,NaN,钞,NaN,NaN,231,230645051A04,230645051


In [15]:
import dtale

In [28]:
dtale.show(df, open_browser=True)

## 测试识别文件列表中的文件类型

In [ ]:
from collections import Counter
from corelibs.process import classify_files_by_category

base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】")
_file_names = list(base_dir.glob('[!~]*.xlsx')) # 找到目录中所有的excel文件（不含子目录）
f,e = classify_files_by_category(_file_names)

print(f)
len([print(k.name + v) for k, v in e.items()])

In [9]:
base_dir = pathlib.Path(r'D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】')
list(base_dir.glob('[!~]*.xlsx')) 


[]

## 测试从账户信息中充实流水数据
（文件夹分析，先分析账户，再分析流水）

In [ ]:
base_dir = pathlib.Path(r'D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】')


## 测试分析银行目录

In [ ]:
from banks.ccb import process_dir_ccb_branch_v2
base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】")
process_dir_ccb_branch_v2(base_dir, OUTPUT_DIR_PATH, 's')

# 执行

## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

In [25]:
from banks.ccb import process_dir_ccb_branch_v1
base_dir = pathlib.Path(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】")
err_files = process_dir_ccb_branch_v1(base_dir, OUTPUT_DIR_PATH, 'a')
print('以下为发生错误而跳过的文件：')
len([print(k.name + v) for k, v in err_files.items()])

分析目录文件:   0%|          | 0/8 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_2_1_watermark.xlsx……建设银行网点流水done
1080860401675667712130255_3_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_4_1_watermark.xlsx……建设银行网点流水done
1080860401675667712130255_5_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_6_1_watermark.xlsx……建设银行网点流水done
1080860401675667712130255_7_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_8_1_watermark.xlsx……建设银行网点流水done
以下为发生错误而跳过的文件：


0

In [17]:
base_dir.__str__()

'D:\\gitcodes\\testdata\\建行(于东林、刘德阳等)【已分析】'

# 杂项

In [14]:
import pandas as pd
df2 = pd.DataFrame({'a':[1,2,3,4],'b':[5,6,7,8],'c':[9,10,11,12]})
df1 = pd.DataFrame({'a':[1,2,1,5],'b':[1,2,1,5],'d':[13,14,15,16]})
print(df1)
print(df2)
pd.merge(df1,df2,left_on='a',right_on='a', how='left', validate='m:1', copy=False)

   a  b   d
0  1  1  13
1  2  2  14
2  1  1  15
3  5  5  16
   a  b   c
0  1  5   9
1  2  6  10
2  3  7  11
3  4  8  12


,a,b_x,d,b_y,c
0,1,1,13,5.0,9.0
1,2,2,14,6.0,10.0
2,1,1,15,5.0,9.0
3,5,5,16,NaN,NaN


In [23]:
df2.drop('a',axis=1)

,b,c
0,5,9
1,6,10
2,7,11
3,8,12


In [ ]:
import pandas as pd
pd.read_excel(r"D:\gitcodes\testdata\建行(于东林、刘德阳等)【已分析】\1080860401675667712130255_1_1_watermark.xlsx")